In [312]:
import spacy
nlp = spacy.load("en_core_web_lg")

import numpy as np
import re
import importlib
import time
from typing import Optional

import ogree_wiki as wiki
importlib.reload(wiki)

<module 'ogree_wiki' from 'c:\\Users\\Admin\\Documents\\IMT Atlantique\\UE -Projet Entreprise-\\OGrEE_NLP\\ogree_wiki.py'>

In [341]:
ACTIONS_DEFAULT = {
                    "ACTION_POSITIVE" : ["make","build","put","place"],
                    "ACTION_NEGATIVE" : ["remove", "delete"], 
                    "ALTERATION" : ["modify", "change","move","set","rename","rotate"]
                    }

ACTIONS_CLI = {
                "ACTION_POSITIVE" : "+",
                "ACTION_NEGATIVE" : "-"
                }

SIMILARITY_PARAMETER = 0.5

KEY_WORDS_DICT = {
            "name" : ["name","called"],
            "position" : ["position", "located","centered"],
            "rotation" : ["rotation","turned","degree"],
            "size" : ["size","dimensions"],
            "template" : ["template"], #TO DO : Rework on template
            "orientation" : ["orientation"],
            "axisOrientation" : ["axisOrientation"],
            "slot" : ["slot"], #same as template
            "color" : ["color"], #Est-ce qu'on rajoute des couleurs genre red, green, grey, ...?
            "unit" :  ["unit"],    # C'est quoi le mieux ? ["t","m","f",'metre', "tilde", "feet"],
            "floorUnit" : ["floorUnit"],
            "side" : ["side"], #Can we just put all the side in this list : it could make our code more resilient
            "temperature" : ["temperature"], #Same here with cold or warm
            "font" : ["font"],
            "attribute" : ["attribute"], #Same as template
            "background" : ["background"],
            "startPosition" : ["startPosition"],
            "endPosition" : ["endPosition"],
            "wall" : ["wall"]
          #  "reserved" : ["reserved"],   
           # "technical" : ["technical"]
            }


In [4]:
def findIndexAction(processed_entry : type(nlp("")), indexAction : int, list_index_entities : list) -> int :
    # identify relations for each entity recognized
    currentIndexes = list_index_entities[::]
    counter = 0
    while counter < 3 :
        for i in range(len(list_index_entities)) :
            currentWord = processed_entry[currentIndexes[i]].head
            currentIndexes[i] = list(processed_entry).index(currentWord)
        counter += 1

    currentIndexesBool = [index == indexAction for index in currentIndexes]
    if sum(currentIndexesBool) >= 2 :
        currentIndexDistance = [(abs(indexAction - list_index_entities[i]),list_index_entities[i]) for i in range(len(currentIndexes)) if currentIndexesBool[i] == True]
        return min(currentIndexDistance)[1]
    elif sum(currentIndexesBool) == 1:
        return list_index_entities[currentIndexesBool.index(True)]
    else :
        raise Exception("Interpretation issue : no relation found, please try again")


In [5]:
def findIndexMainSubject(processed_entry : type(nlp("")), indexAction : int, dictioIndexKeyWords : dict) -> int :

    counter = 0
    currentIndexes = {index:index for index,parameter in dictioIndexKeyWords.items() if parameter not in ACTIONS_DEFAULT.keys()}
    currentWords = {index:processed_entry[index] for index,parameter in dictioIndexKeyWords.items() if parameter not in ACTIONS_DEFAULT.keys()}
    while (not indexAction in currentIndexes.values()) and counter < 3 :
        currentWords = {originIndex : processed_entry[currentIndex].head for originIndex,currentIndex in currentIndexes.items()}
        currentIndexes = {originIndex : list(processed_entry).index(currentWords[originIndex]) for originIndex,_ in currentIndexes.items()}
        counter += 1

    if counter == 3 :
        raise Exception("Did not find the main action")
    
    if list(currentIndexes.values()).count(indexAction) != 1 :
        listIndexesRemaining = [originIndex for originIndex,currentIndex in currentIndexes.items() if currentIndex == indexAction and originIndex > indexAction]
        return listIndexesRemaining[0]
    else :
        return [originIndex for originIndex,currentIndex in currentIndexes.items() if currentIndex == indexAction][0]


In [6]:
def name(processed_entry : type(nlp("")), dictioEntities : dict, indexesMain : list) -> int :

    def findClose(processed_entry : type(nlp("")), index : int) -> (int|None) :
        if index +1 <= len(processed_entry)-1 :
            if str(processed_entry[index+1]).isupper() or processed_entry[index+1].pos_ == "PROPN" :
                return index+1
        if 0 <= index -1 :
            if str(processed_entry[index-1]).isupper() or processed_entry[index-1].pos_ == "PROPN" :
                return index-1
        return None
    

    def findAttachedEntity(processed_entry : type(nlp("")), index : int) -> (int|None) : 
        counter = 0
        isFound = False
        currentIndex = index
        currentWord = processed_entry[currentIndex]
        while (not isFound) and counter < 3 :
            currentWord = processed_entry[currentIndex].head
            currentIndex = list(processed_entry).index(currentWord)
            if str(currentWord) in dictioEntities.values() :
                return currentIndex
            if currentWord.pos_ == "VERB" and currentIndex == indexesMain[1] : 
                return indexesMain[0]
            counter += 1
        return None
    
    EXPLICIT =  KEY_WORDS_DICT["name"]
    IMPLICIT = ["current","main"]

    dictioEntityNames = {}
        
    for index,entity in dictioEntities.items() :
    # if the name if right beside the entity
        resultIndex = findClose(processed_entry, index)
        if resultIndex != None :
            dictioEntityNames[index] = resultIndex

    if len(dictioEntityNames) < len(dictioEntities) : # if not all names found

        for index,token in enumerate(processed_entry) : # look for keyword

            if str(processed_entry[index]).isupper() or processed_entry[index].pos_ == "PROPN" :
                indexAttachedEntity = findAttachedEntity(processed_entry,index)
                if not (indexAttachedEntity in dictioEntityNames or indexAttachedEntity == None) :
                    dictioEntityNames[indexAttachedEntity] = index
            
            # if the token is a synonym of called
            if sum([token.similarity(nlp(word)[0]) > SIMILARITY_PARAMETER for word in EXPLICIT]) >= 1 :
                resultIndex = findClose(processed_entry, index)
                if resultIndex != None :
                    indexAttachedEntity = findAttachedEntity(processed_entry,index)
                    if not (indexAttachedEntity in dictioEntityNames or indexAttachedEntity == None) :
                        dictioEntityNames[indexAttachedEntity] = resultIndex


    if len(dictioEntityNames) < len(dictioEntities) : # if still not all names found
        pass
    # TODO : get current names

    # TODO : check for implicit words now
    
    return dictioEntityNames

In [40]:
def position(processed_entry : type(nlp("")), index : int, lastKeyWordIndex : int, nextKeyWordIndex : int,entity = "") -> Optional[list] :
    # search for keywords to the right and left to split whole sentence into parts where to look for param entries
    
    next_words = processed_entry[index+1:nextKeyWordIndex]
    last_words = processed_entry[lastKeyWordIndex:index]
    position_list = re.findall("[-]*[0-9]+[.]*[0-9]*", " ".join([str(token) for token in next_words]))
    if not (len(position_list) == 2 or (len(position_list) == 3 and entity == "rack") or (len(position_list) ==1 and entity =="device")):
        position_list_left = re.findall("[-]*[0-9]+[.]*[0-9]*", " ".join([str(token) for token in last_words]))
        if not (len(position_list) == 2 or (len(position_list) == 3 and entity == "rack")) :
            raise Exception("Wrong location format entered")
        else :
            position_list = position_list_left
    result = [float(coord) for coord in position_list] if position_list else None
    return result[0] if entity == "device" else result

In [44]:
text = nlp("build a device named RACK1 at the postion 10 10 ")
print(position(text,7,0,len(text),""))

[10.0, 10.0]


In [9]:
def axisOrientation(processed_entry : type(nlp("")), index : int, lastKeyWordIndex : int, nextKeyWordIndex : int,entity="") -> Optional[list] :
    # We search the axis Orientation
    #We slit the sentence in two parts
    next_words = processed_entry[index+1:nextKeyWordIndex]
    last_words = processed_entry[lastKeyWordIndex:index]
    #An axis Orientation  can be any combinason of [+/-]x[+/-]y. eg: +x+y or -x+y
    print(" ".join([str(token) for token in next_words]))
    axisOrientationList =  re.findall(" (-x|\+[ *]x)[ *](-y|\+[ *]y)", " ".join([str(token) for token in next_words]))
    #If we didn't find an axis orientation, we need to seek it in the second part of the sentence
    if len(axisOrientationList) ==0 :
        axisOrientationList = re.findall("(-x|\+[ *]x)[ *](-y|\+[ *]y)", " ".join([str(token) for token in last_words]))
        #If we still didn't find it, we need to raise an error
        if len(axisOrientationList) ==0 :
            raise Exception("Wrong axisOrientation format entered")
    
    #We need to raise an error if we find only one coordonate
    if len(axisOrientationList[0]) < 2:
        raise Exception("Only one coordinate was found")
    #Even if several axis Orientations where found, we always take the first one
    return axisOrientationList[0][0].replace(" ", "") + axisOrientationList[0][1].replace(" ", "")


In [55]:
def unit(processed_entry : type(nlp("")), index : int, lastKeyWordIndex : int, nextKeyWordIndex : int,entity="") -> Optional[list] :
    #This function help us to find the unit of the floor
    #We slit the sentence in two parts
    next_words = processed_entry[index+1:nextKeyWordIndex]
    last_words = processed_entry[lastKeyWordIndex:index]
    #An unit can be m, t , f, meters, tildes, feets
    unitList = re.findall(r'\bt\b|\bf\b|\bm\b|\bu\b'," ".join([str(token) for token in next_words]))
    unitList += re.findall("metre|feet|tilde|unit"," ".join([str(token) for token in next_words]))
    #If we didn't find any unit, we need to seek it in the second part of the sentence
    if len(unitList) ==0:
        unitList = re.findall(r'\bt\b|\bf\b|\bm\b|\bu\b'," ".join([str(token) for token in last_words]))
        unitList += re.findall("metre|feet|tilde|unit"," ".join([str(token) for token in last_words]))
        
        #We should have found an unit
        if len(unitList) == 0:
            raise Exception("There wasn't any unit in the sentence")
    
    #If there are many unit, we choose the first one
    finalUnit = unitList[0]
    match unitList[0]:
        case "metre":
            finalUnit = "m"
        case "tilde":
            finalUnit = "t"
        case "feet":
            finalUnit = "f"
        case "unit":
            finalUnit ="u"
    return finalUnit

In [56]:
test = nlp("Create a room in f")
print(unit(test,3,0,len(test)))

f


In [188]:
def color(processed_entry : type(nlp("")), index : int, lastKeyWordIndex : int, nextKeyWordIndex : int,entity="") -> Optional[list] :
    #This function seek the hexadecimal color next to the word color (or one of its synonym)
    #We slit the sentence in two parts
    next_words = processed_entry[index+1:nextKeyWordIndex]
    last_words = processed_entry[lastKeyWordIndex:index]
    #A color should be in hexadecimal ==> MAYBE A NAME LATER ??
    colorList = re.findall(r'#[ *][A-Z0-9a-z]{6}\b'," ".join([str(token) for token in next_words]))

    #If we didn't find a color after the word color, we need to look before
    if len(colorList) ==0:
        colorList = re.findall(r'#[ *][A-Z0-9a-z]{6}\b'," ".join([str(token) for token in last_words]))

        #If there isn't any color, we need to raise an execption
        if len(colorList) == 0: 
            raise Exception("There wasn't any color")
        
    #If there are several colors, we choose the first one
    return colorList[0].replace(" ", "")

In [233]:
def font(processed_entry : type(nlp("")), index : int, lastKeyWordIndex : int, nextKeyWordIndex : int,entity="") ->  Optional[list] :
    #This function find the font of a label.
    #We slit the sentence in two parts
    next_words = processed_entry[index+1:nextKeyWordIndex]
    last_words = processed_entry[lastKeyWordIndex:index]
    print("next_word : ", next_words)
    findList = re.findall(r'#[ *][A-Z0-9a-z]{6}\b'," ".join([str(token) for token in next_words]))
    findList += re.findall('italic|bold'," ".join([str(token) for token in next_words]))
    if len(findList) == 0:
        findList = re.findall(r'#[ *][A-Z0-9a-z]{6}\b'," ".join([str(token) for token in last_words]))
        findList += re.findall('italic | bold'," ".join([str(token) for token in last_words]))
        if len(findList) == 0: 
            raise Exception("There wasn't any argument")
    
    #We return the list of the argument associate to the font
    return findList


In [234]:
test = nlp("create a label with a font #154741 and in bold")
print(font(test,5,0,len(test)))

next_word :  #154741 and in bold
['# 154741', 'bold']


In [201]:
def background(processed_entry : type(nlp("")), index : int, lastKeyWordIndex : int, nextKeyWordIndex : int,entity="") ->  Optional[list] :
    #This function find the color of the background with the help of the function color
    return color(processed_entry,index,lastKeyWordIndex,nextKeyWordIndex,entity="")

In [288]:
def slot(processed_entry : type(nlp("")), index : int, lastKeyWordIndex : int, nextKeyWordIndex : int,entity="") ->  Optional[list] :
    #The code for slot is the same as template --> Works well if the name is at the rigt of template and if there is a verb like called, named,...
    return template(processed_entry, index, lastKeyWordIndex,nextKeyWordIndex,entity)

In [289]:
def attribute(processed_entry : type(nlp("")), index : int, lastKeyWordIndex : int, nextKeyWordIndex : int,entity="") ->  Optional[list] :
    #The code for attribute is the same as template --> Works well if the name is at the rigt of template and if there is a verb like called, named,...
    return template(processed_entry, index, lastKeyWordIndex,nextKeyWordIndex,entity)

In [228]:
def startPosition(processed_entry : type(nlp("")), index : int, lastKeyWordIndex : int, nextKeyWordIndex : int,entity = "") -> Optional[list] :
    # search for keywords to the right and left to split whole sentence into parts where to look for param entries
    
    next_words = processed_entry[index+1:nextKeyWordIndex]
    last_words = processed_entry[lastKeyWordIndex:index]
    position_list = re.findall("[-]*[0-9]+[.]*[0-9]*", " ".join([str(token) for token in next_words]))
    #A separator has always x y as startPosition
    if len(position_list) > 2:
        #We found the startPosition but also the position of another entity. Ex : startPos 10 25 and 10 87 as endPos
        #Here position_list == [10,25,10,87] ==> We must only take the 2 first cordonates
        position_list = [position_list[0],position_list[1]]
    
    if not len(position_list) == 2:
        position_list_left = re.findall("[-]*[0-9]+[.]*[0-9]*", " ".join([str(token) for token in last_words]))
        #Same here
        if len(position_list_left) > 2:
            position_list_left = [position_list_left[-2],position_list_left[-1]]
        if not (len(position_list_left) == 2) :
            raise Exception("Wrong location format entered")
        else :
            position_list = position_list_left
    result = [float(coord) for coord in position_list] if position_list else None
    return result

In [242]:
def endPosition(processed_entry : type(nlp("")), index : int, lastKeyWordIndex : int, nextKeyWordIndex : int,entity = "") -> Optional[list] :
    return startPosition(processed_entry,index,lastKeyWordIndex,nextKeyWordIndex)

In [271]:
def wall(processed_entry : type(nlp("")), index : int, lastKeyWordIndex : int, nextKeyWordIndex : int,entity = "") -> Optional[list] :
    #This function find the type of wall of the separator.
    #We slit the sentence in two parts
    next_words = processed_entry[index+1:nextKeyWordIndex]
    last_words = processed_entry[lastKeyWordIndex:index]
    print("wall : ", next_words)
    findList = re.findall('wireframe|plain'," ".join([str(token) for token in next_words]))
    if len(findList) == 0:
        findList += re.findall('wireframe|plain'," ".join([str(token) for token in last_words]))
        if len(findList) == 0: 
            raise Exception("There wasn't any argument")
    
    #We return the list of the argument associate to the font
    return findList[0]

In [277]:
test = nlp("create a separator with a endPosition of 0 10 and 15 47 as startPosition and a wall in wireframe ")
print(wall(test,12,0,len(test)))

wall :  startPosition and a wall in wireframe
wireframe


In [244]:
test = nlp("create a separator with a endPosition of 0 10 and 15 47 as startPosition")
print(startPosition(test,len(test),0,len(test)))
test = nlp("create a separator with a endPosition of 0 10 and 15 47 as startPosition")
print(endPosition(test,4,0,len(test)))

[15.0, 47.0]
[0.0, 10.0]


In [204]:
test = nlp("create a label with a background in #ABCdEF")
print("background : ", background(test,5,0,len(test)))

background :  #ABCdEF


In [11]:
def temperature(processed_entry : type(nlp("")), index : int, lastKeyWordIndex : int, nextKeyWordIndex : int,entity="") -> Optional[list] :
    #This function helps to find if its cold or warm
    #We slit the sentence in two parts
    next_words = processed_entry[index+1:nextKeyWordIndex]
    last_words = processed_entry[lastKeyWordIndex:index]
    
    #A temperature is either cold or warm 
    tempList = re.findall("cold|warm", " ".join([str(token) for token in next_words]))
    
    #If we didn't find it after the word temperature, we need to seek before
    if len(tempList) ==0:
        tempList = re.findall("cold|warm", " ".join([str(token) for token in last_words]))

        #If neither cold or warm was found we need to raise an Exception
        if len(tempList) == 0:
            raise Exception("Neither cold or warm was found in the sentence")

    #If there are several temperature, we choose the first one
    return tempList[0]
    


In [12]:
def side(processed_entry : type(nlp("")), index : int, lastKeyWordIndex : int, nextKeyWordIndex : int,entity="") -> Optional[list] :
    #We slit the sentence in two parts
    next_words = processed_entry[index+1:nextKeyWordIndex]
    last_words = processed_entry[lastKeyWordIndex:index]

    #A side is either front, rear, frontflipped,rearflipped 
    sideList = re.findall("front | rear | frontflipped | rearflipped", " ".join([str(token) for token in next_words]))
    #if a side was not found after the word, we need to seek after
    if len(sideList) == 0:
        sideList = re.findall("front | rear | frontflipped | rearflipped", " ".join([str(token) for token in last_words]))

        #If the side wasn't after the word side, there is an error
        if len(sideList) == 0: 
            raise Exception( "there wasn't any side in the sentence")

    #If there are several sides, we take the first one
    return sideList[0].replace(" ","")


In [13]:
def floorUnit(processed_entry : type(nlp("")), index : int, lastKeyWordIndex : int, nextKeyWordIndex : int,entity="") -> Optional[list] :
    un =unit(processed_entry, index, lastKeyWordIndex, nextKeyWordIndex)
    if un.replace(" ", "") == "u":
        raise Exception ("A floor Unit can't have u as unit")
    return un 

In [14]:
def slot(processed_entry : type(nlp("")), index : int, lastKeyWordIndex : int, nextKeyWordIndex : int,entity="") -> Optional[list] :
    #Find a slot is equivalent at finding a name, the slot name
    pass
    

In [140]:
testNlp = nlp("create a room called ROOM1 with a  temperature warm with an orientation of -x -y  in u with #abD874 #ABDDDD as color with rear as side   ")

print("axisOrientation : ", axisOrientation(testNlp, 0,0,len(testNlp) -1).replace(" ", ""))
print("Unit : ", unit(testNlp,0,0,len(testNlp) -1))
print("Color : ", color(testNlp,22,0,len(testNlp) -1))
print("Temperature", temperature(testNlp,12,0,len(testNlp) -1))
print("Side : ", side(testNlp, 20,0,len(testNlp) -1) )

a room called ROOM1 with a   temperature warm with an orientation of -x -y   in u with # abD874 # ABDDDD as color with rear as side
axisOrientation :  -x-y
Unit :  u
next_words ABDDDD as color with rear as side
last_words create a room called ROOM1 with a  temperature warm with an orientation of -x -y  in u with #abD874
Color :  #abD874
Temperature warm
Side :  rear


In [332]:
def orientation(processed_entry : type(nlp("")), index : int, lastKeyWordIndex : int, nextKeyWordIndex : int, entity="") -> Optional[float] :
    next_words = processed_entry[index+1:nextKeyWordIndex]
    last_words = processed_entry[lastKeyWordIndex:index]
    orientation_list = re.findall("N|S|W|E|NW|NE|SW|SE|ESE|WNW|NNW|NNE|ENE|WSW|SSW|SSE", " ".join([str(token) for token in next_words]))
    if not len(orientation_list) == 1:
        orientation_list = re.findall("N|S|W|E|NW|NE|SW|SE|ESE|WNW|NNW|NNE|ENE|WSW|SSW|SSE", " ".join([str(token) for token in last_words]))
        if not len(orientation_list) == 1:
            raise Exception("None orientation was found")
    
    return orientation_list[0]

In [331]:
def rotation(processed_entry : type(nlp("")), index : int, lastKeyWordIndex : int, nextKeyWordIndex : int, entity="") -> Optional[float] :
    next_words = processed_entry[index+1:nextKeyWordIndex]
    last_words = processed_entry[lastKeyWordIndex:index]
    
    isRotationNegative = False

    rotation_list = re.findall("([-]*[0-9]+[.]*[0-9]*){1,3}", " ".join([str(token) for token in next_words]))
    isRotationNegative = re.search("counter.*clockwise", "".join([str(token) for token in next_words]))
    if not len(rotation_list) == 1 and not len(rotation_list) == 3 :
        rotation_list_left = re.findall("[-]*[0-9]+[.]*[0-9]*", " ".join([str(token) for token in last_words]))
        isRotationNegative = re.search("counter.*clockwise", "".join([str(token) for token in next_words]))
        if not len(rotation_list_left) == 1 :
            raise Exception("Wrong rotation format entered")
        else :
            rotation_list = rotation_list_left
    rotation_list = [float(token) for token in rotation_list]
    if (entity == "rack" or entity == "corridor") and len(rotation_list) !=3:
        raise Exception("The rotation of a rack or a corridor should have a size 3")
    if entity != "rack" and entity !="corridor" and len(rotation_list) !=1:
        raise Exception("Should have only 1 argument for size")
    rotationFinal = rotation_list[0] if len(rotation_list) == 1 else rotation_list
    rotationFinal = -rotationFinal if isRotationNegative else rotationFinal
    return rotationFinal

In [73]:
text = nlp("create a corridor with a rotation of 90 90 90")
print(rotation(text,4,0,len(text),"corridor"))

next_word :  rotation of 90 90 90
rotation_list :  ['90', '90', '90']
[90.0, 90.0, 90.0]


In [356]:
def size(processed_entry : type(nlp("")), index : int, lastKeyWordIndex : int, nextKeyWordIndex : int, entity="") -> Optional[list] :
    #it only works for regular size not sizeXY or sizeU
    next_words = processed_entry[index+1:nextKeyWordIndex]
    last_words = processed_entry[lastKeyWordIndex:index]

    size_list = re.findall("[-]*[0-9]+[.]*[0-9]*", " ".join([str(token) for token in next_words]))
    print(size_list)
    if not (len(size_list) == 3) and not( entity =="pillar" and len(size_list) == 2 ) and not(len(size_list)==1 and entity == "device"):
        size_list_left = re.findall("[-]*[0-9]+[.]*[0-9]*", " ".join([str(token) for token in last_words]))
        if not (len(size_list) == 3) and not( entity =="pillar" and len(size_list) == 2 ) and not(len(size_list)==1 and entity == "device"):
            raise Exception("Wrong location format entered")
        else :
            size_list = size_list_left
    result = [float(coord) for coord in size_list] if size_list else None
    return result[0] if entity == "device" else result

In [357]:
text = nlp("create the pillar with 0 0  as size")
print("Size  : ", size(text, 0,0,len(text)-1,"pillar"))

['0', '0']
Size  :  [0.0, 0.0]


In [215]:
def template(processed_entry : type(nlp("")), index : int, lastKeyWordIndex : int, nextKeyWordIndex : int, entity="") -> Optional[str] :
    #The template is the defined by a name, it's the same structure as for Name() but simplified since there is less convuloted wording
    #Works well if the name is at the right of the word template and if called, named, ... is used
    def findClose(indexToken : int) -> Optional[int] :
        if indexToken +1 <= len(processed_entry)-1 :
            if str(processed_entry[indexToken+1]).isupper() or processed_entry[indexToken+1].pos_ == "PROPN" :
                return indexToken+1
        if 0 <= indexToken -1 :
            if str(processed_entry[indexToken-1]).isupper() or processed_entry[indexToken-1].pos_ == "PROPN" :
                return indexToken-1
        return None

    def findAttachedEntity(indexToken : int) -> (bool) : 
        counter = 0
        isFound = False
        currentIndex = indexToken
        currentWord = processed_entry[currentIndex]
        while (not isFound) and counter < 3 :
            currentWord = processed_entry[currentIndex].head
            currentIndex = list(processed_entry).index(currentWord)
            if str(currentWord) in KEY_WORDS_DICT["template"] : #Peux être faut il passer aux Lemma ?
                return True
            counter += 1
        return False
    
    EXPLICIT =  KEY_WORDS_DICT["name"]

    # if the name if right beside the entity
    resultIndex = findClose(index)
    if resultIndex != None :
        return processed_entry[resultIndex]
    else: #look in the NProp if one is linked to Template
        for indexTemp,tokenTemp in enumerate(processed_entry) : # look for keyword
            if str(processed_entry[indexTemp]).isupper() or processed_entry[indexTemp].pos_ == "PROPN" :
                indexAttachedEntity = findAttachedEntity(indexTemp)
                if findAttachedEntity(indexTemp) :
                    return tokenTemp
            
            """
            # if the token is a synonym of called
            if sum([tokenTemp.similarity(nlp(word)[0]) > SIMILARITY_PARAMETER for word in EXPLICIT]) >= 1 :
                resultIndex = findClose(indexTemp)
                if resultIndex != None :
                    indexAttachedEntity = findAttachedEntity(indexTemp)
                    if indexAttachedEntity != None :
                        resultIndex = indexAttachedEntity
                else {
                    return resultIndex
                }
            """

    return None

In [214]:
test = nlp("Create a room with a template called FUCKER")

print(template(test,2,2,len(test)))

FUCKER


In [24]:
pip install textblob

     ---------------------------------------- 0.0/636.8 kB ? eta -:--:--
     ----- --------------------------------- 92.2/636.8 kB 1.8 MB/s eta 0:00:01
     --------- ---------------------------- 163.8/636.8 kB 2.0 MB/s eta 0:00:01
     ---------------- --------------------- 276.5/636.8 kB 2.1 MB/s eta 0:00:01
     ------------------------- ------------ 419.8/636.8 kB 2.0 MB/s eta 0:00:01
     ----------------------------- -------- 491.5/636.8 kB 2.2 MB/s eta 0:00:01
     -------------------------------------  634.9/636.8 kB 2.2 MB/s eta 0:00:01
     -------------------------------------- 636.8/636.8 kB 2.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     ---- ----------------------------------- 0.2/1.5 MB 3.5 MB/s eta 0:00:01
     ------- -------------------------------- 0.3/1.5 MB 3.5 MB/s eta 0:00:01
     --------- ------------------------------ 0.4/1.5 MB 2.9 MB/s eta 0:00:01
     ------------- -------------------------- 0.5/1.5 MB 3.0 M


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [36]:
from textblob import TextBlob
 
a = "cmputr"           # incorrect spelling
print("original text: "+str(a))
 
b = TextBlob(a)
 
# prints the corrected spelling
print("corrected text: "+str(b.correct()))

test = nlp("crete a site calld SITE 0 0 0")

for word in test:
    print("Old word : " , word)
    b = TextBlob(str(word))
    print("corrected text: "+str(b.correct()))

original text: cmputr
corrected text: computer
Old word :  crete
corrected text: create
Old word :  a
corrected text: a
Old word :  site
corrected text: site
Old word :  calld
corrected text: called
Old word :  SITE
corrected text: SITE
Old word :  0
corrected text: 0
Old word :  0
corrected text: 0
Old word :  0
corrected text: 0


In [381]:
"""This module contains static tools for the different classes"""

import re

def create(typeOfObject : str, parameters : dict) -> str:
    """Creates an object from given parameters"""
    if not isConform(parameters,typeOfObject):
        raise ValueError("The parameters given are invalid for the object of type :" + typeOfObject)
    return "+{}:".format(typeOfObject) + "@".join([str(parameters[key]) for key in parameters.keys()])

def delete(typeOfObject : str, parameters : dict) -> str:
    """Deletes an object from given parameters
    We consider that the object exist with this name"""
    return "-{}".format(parameters["name"])

def setName(oldName, newName : str) -> str:
    """Changes the name of an object, and ensures to keep it coherent"""
    oldNameParent = getParentName(oldName)
    newNameParent = getParentName(newName)
    if oldNameParent != newNameParent:
        raise ValueError("The new name is invalid because it does not respect the convention of the parent name")
    return "{}:name={}".format(oldName, newName)

def setAttribute(name : str, attributeName : str, attributeArgument) -> str:
    """Modifies an attribute of an object with a given argument"""
    if attributeName in CONFORMITY_CHECK:
        if not CONFORMITY_CHECK[attributeName](attributeArgument):
            raise ValueError("The new {} is invalid because the value {} is incorrect".format(attributeName,attributeArgument))
    return "{}:{}={}".format(name,attributeName,attributeArgument)

def createAttribute(name : str, attributeName : str, attributeArgument) -> str:
    """Adds an attribute to an object with a given argument"""
    return "{}.{}={}".format(name, attributeName,attributeArgument)

def isConform(parameters : dict, entity="") -> bool:
    """Verifies that the parameters given are conform thanks to multiple verification
    functions, which are called if the parameter is in the parameters"""
    if (CONFORMITY_CHECK == {}):
        raise NotImplementedError("There are no implemented conformity checks")
    return all([CONFORMITY_CHECK[attribute](parameters[attribute],entity) for attribute in parameters.keys()])

#region:Tests
def isListOfNumbers(lst : list) -> bool:
    """Verifies that the list given in argument only contains numbers"""
    n = len(lst)
    k = 0
    verified = True
    while verified and k < n:
        verified = verified and (type(lst[k]) in [float, int])
        k += 1
    return verified

def isNameConform(name : str, entity="") -> bool:
    """Verifies that the argument is a string. This function will potentially be changed if there are more
    specifications for the name"""
    return type(name) == str

def isPositionConform(position : list,entity="") -> bool:
    """Verifies that the list given in argument represents a position"""
    match entity:
        case 'device':
            return type(position) in [int,float]
        case 'rack':
            return (len(position) == 2 or len(position) ==3) and isListOfNumbers(position)
        case 'corridor':
            return (len(position) == 2 or len(position) ==3) and isListOfNumbers(position)
        case other:
            return len(position) == 2 and isListOfNumbers(position)


def isRotationConform(rotation : float|list,entity="") -> bool:
    """Verifies that the argument is a number"""
    match entity:
        case 'rack':
            return type(rotation)==list and len(rotation) == 3 and isListOfNumbers(rotation)
        case 'corridor':
            return  type(rotation)==list and len(rotation) == 3 and isListOfNumbers(rotation)
        case other:
            return type(rotation) in [float,int]
    


def isSizeConform(size : list,entity="") -> bool:
    """Verifies that the list given in argument represents a size"""
    match entity:
        case 'device':
            return type(size) in [int,float]
        case 'pillar':
            return len(size) == 2 and isListOfNumbers(size)
        case other:
            return len(size) == 3 and isListOfNumbers(size)
    

def isAxisOrientationConform(axis : str,entity="") -> bool:
    """Verifies that the axis orientation is conform"""
    return bool(re.compile(r"[+-]x[+-]y").match(axis))
    
def isFloorUnitConform(floorUnit : str,entity="") -> bool:
    """Verifies that the floor unit is conform"""
    return floorUnit.replace(" ","") in ["t","f","m"]

def isUnitConform(floorUnit : str,entity="") ->bool:
    """Verifies that the floor unit is conform"""
    return floorUnit.replace(" ","") in ["t","f","m","u"]

def isColorConform(color : str,entity="") -> bool:
    """Verifies that the string given in argument is an hexadecimal color"""
    return bool(re.compile(r"#[0-9A-F]{6}").match(color))


def isOrientationConform(orientation : str,entity="") -> bool:
    """Verifies that the string given in argument is a valid orientation"""
    return orientation in ["N","S","W","E","NW","NE","SW","SE","ESE"
                           "WNW","NNW","NNE","ENE","WSW","SSW","SSE"]

def isSlotConform(slot : str,entity="") -> bool:
    return type(slot) == str

def isTemplateConform(temp :  str,entity="") -> bool:
    return type(teùmp) == str

def isSideConform(side : str,entity="") -> bool:
    return side in ["front", "rear", "frontflipped", "rearflipped"]

def isTemperatureConform(temperature : str,entity="") -> bool:
    return temperature in ["cold", "warm"]

def isWallConform(wall : str, entity=""):
    return bool(re.compile("plain|wireframe").match(wall))

print(isWallConform("plain"))
#endregion

#region:Helpers
def parametersToString(parameters : list) -> list:
    """Returns a list with the type of the parameters in the list changed to string"""
    return [str(parameter) for parameter in parameters]

def getParentName(completeName : str) -> str:
    """Returns the name of the parent object. It reverses the name, then splits it using dot as separator, and only
    gets the first part of the name, which is put back in order."""
    return "".join(reversed(completeName)).split("/",1)[-1][::-1]
#endregion


CONFORMITY_CHECK = {"name" : isNameConform, "orientation" : isOrientationConform, "position" : isPositionConform,
                    "rotation" : isRotationConform, "size" : isSizeConform, "axisOrientation" : isAxisOrientationConform,
                    "floorUnit" : isFloorUnitConform, "color" : isColorConform, "template" : isTemplateConform,
                    "slot" : isSlotConform, "side": isSideConform,"temperature":isTemperatureConform,"unit":isUnitConform,
                    "wall" : isWallConform, "startPosition" : isPositionConform, "endPosition" : isPositionConform,
                    "attribute":isTemplateConform
                    }

if __name__ == "__main__":
    print(create("site",{"name" : "P/BASIC", "orientation":"WSW"}))
    print(setAttribute("P/BASIC","orientation","WSW"))




True
+site:P/BASIC@WSW
P/BASIC:orientation=WSW


In [391]:
def main() -> str :
    FINAL_INSTRUCTION = ""

    ENTITIES_FULL_NAME = {"entity" : list(wiki.ENTITIES.keys())}

    # TODO : add already existing entity names
    KEY_WORDS_ALL = {**ENTITIES_FULL_NAME,  **KEY_WORDS_DICT}
    # for parameter in KEY_WORDS_ALL.keys() :
    #     for word in KEY_WORDS_ALL[parameter] :
    #         print(word,nlp(word)[0].pos_)
    KEY_WORDS_ENTRY = {}

    natural_entry = input("Enter a prompt. Please follow the instructions given.\n")
    processed_entry = nlp(natural_entry)

    # we detect key words in the sentence given and put them into KEY_WORDS_ENTRY
    for index,token in enumerate(processed_entry) :
        matching_list = []
        if token.pos_ == "VERB" and str(token) == token.lemma_ : # infinitive verb
            for parameter in ACTIONS_DEFAULT.keys() :
                similarity = max([token.similarity(nlp(word)[0]) for word in ACTIONS_DEFAULT[parameter]])
                matching_list.append((similarity,parameter))
        elif token.pos_ in ["NOUN","ADP","VERB"]:
            for parameter in KEY_WORDS_ALL.keys() : 
                similarity = max([token.similarity(nlp(word)[0]) for word in KEY_WORDS_ALL[parameter]])
                matching_list.append((similarity,parameter))
        else :
            continue
        match = max(matching_list)
        if match[0] > SIMILARITY_PARAMETER :
            KEY_WORDS_ENTRY[index] = match[1]

    print(KEY_WORDS_ENTRY)

    # test detection
    list_key_param = list(KEY_WORDS_ENTRY.values())
    count_action = 0
    for action_type in ACTIONS_DEFAULT.keys() :
        count_action += list_key_param.count(action_type)

    if count_action != 1 :
        raise Exception("Action not detected")
    else :
        global INDEX_ACTION
        global INDEX_MAIN_SUBJECT
        INDEX_ACTION = [index for index,keyword in KEY_WORDS_ENTRY.items() if keyword in ACTIONS_DEFAULT.keys()][0]
        INDEX_MAIN_SUBJECT = findIndexMainSubject(processed_entry, INDEX_ACTION, KEY_WORDS_ENTRY)  
    print("index main : ",INDEX_MAIN_SUBJECT)

    dictEntities = {index : str(processed_entry[index]) for index,keyword in KEY_WORDS_ENTRY.items() if keyword == "entity"}
    print(dictEntities)
    dictioEntityNames = name(processed_entry, dictEntities, [INDEX_MAIN_SUBJECT, INDEX_ACTION])
    print("names : ",dictioEntityNames)

    # TODO : match cases (main = entity/parameter, verb = +/alteration...)
    print("KEY_WORDS_ENTRY[INDEX_MAIN_SUBJECT]", KEY_WORDS_ENTRY[INDEX_MAIN_SUBJECT])
    if KEY_WORDS_ENTRY[INDEX_MAIN_SUBJECT] == "entity" :
          # we do the processes related to each parameter
        dictioEntityParameters = wiki.makeDictParam(str(processed_entry[INDEX_MAIN_SUBJECT]))
        dictioEntityParameters["name"] = str(processed_entry[dictioEntityNames[INDEX_MAIN_SUBJECT]])
        allEntryItems = KEY_WORDS_ENTRY.items()
        for counter,(index,parameter) in enumerate(allEntryItems) :
            if (parameter in KEY_WORDS_DICT.keys()) and (parameter in dictioEntityParameters.keys()) and parameter != "name":
                lastKeyWordIndex = 0 if counter == 0 else list(allEntryItems)[counter-1][0]
                nextKeyWordIndex = len(processed_entry) if counter == len(allEntryItems)-1 else list(allEntryItems)[counter+1][0]
                # get the parameter value
                parameterValue = globals()[parameter](processed_entry, index, lastKeyWordIndex, nextKeyWordIndex,dictEntities[INDEX_MAIN_SUBJECT])
                dictioEntityParameters[parameter] = parameterValue # store the value
        if KEY_WORDS_ENTRY[INDEX_ACTION] == "ACTION_POSITIVE" :
            print("dictioEntityParameters :", dictioEntityParameters)
            dictFormate = createDict(dictEntities[INDEX_MAIN_SUBJECT],dictioEntityParameters)
            print("dictFormate :", dictFormate)
            command = create(dictEntities[INDEX_MAIN_SUBJECT],dictFormate)
            print("command :",command)
            # TODO : call the class method

        if KEY_WORDS_ENTRY[INDEX_ACTION] == "ACTION_NEGATIVE" :
            print("dictioEntityParameters :", dictioEntityParameters)
            command = delete(dictEntities[INDEX_MAIN_SUBJECT],dictioEntityParameters)
            print("command : ", command)
           # TODO : call the class method

        if KEY_WORDS_ENTRY[INDEX_ACTION] == "ALTERATION" :
            print(dictioEntityParameters)
            # TODO : understand what are the changes and its consequences
            pass

    if KEY_WORDS_ENTRY[INDEX_MAIN_SUBJECT] in KEY_WORDS_DICT.keys() :
        # TODO : seek the value to be changed
        pass


    
    # if seeking the name for the main entity, pass the indexaction as parameter
    # if no name found, check the type of action : if +, a name is needed, otherwise not necessarily

    # check if parameters were not given

    # return KEY_WORDS_ENTRY

In [395]:
text0 = "create a site called SITE with an orientation of E"
text = "create the building called IMT at the position 0 0 rotated 90 and a size of 10*10*10 "
test2 = """create a room called ROOM1 with a position of 4 14 a rotation of 10
an axisOrientation of -x -y  a size of 10 10 10 with a floorUnit in m """
test3 = """build a rack named RACK1 at the position 10 -10 50 in unit tildes with a rotation of [0,0,180] and a size of 10 20 30"""
test4 = """ construct a device named DEVICE1 at the position 4 at the size 45"""
test5 = """create a corridor called CORR1 at the position 0 0 with metre as unit a rotation of 0 0 0 a size of 10*10*10 and a warm temperature"""
test6 = """create a tag named TAG1 with the color #154987"""
test7 = """create a pillar called P1 at the position 10 10 with a size of 10 50 and a rotation of 10"""
#PB AVEC MOT WIREFRAME --> Template
test7 = """construct a separator called SEPA1 with wall in plain and a startPosition of 10 25 and a endPosition 54 78 """
test8 = """build a pillar called P1 centered at 10 21 with a size of 15 78 and a rotation of 87"""
#PB because attribute is understood as a name
test9 = """create a label named ATTRIBUTE"""
test10 = """delete the site called SITE"""
test11 = """delete the building  called IMT"""
test12 = """delete the room called ROOM1"""
test13 = """delete the rack called RK1"""
main()

C:\Users\Admin\AppData\Local\Temp\ipykernel_10920\1016861164.py:25: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  similarity = max([token.similarity(nlp(word)[0]) for word in KEY_WORDS_ALL[parameter]])


{0: 'ACTION_NEGATIVE', 2: 'entity', 3: 'name'}
index main :  2
{2: 'rack'}
names :  {2: 4}
KEY_WORDS_ENTRY[INDEX_MAIN_SUBJECT] entity
dictioEntityParameters : {'name': 'RK1', 'position': None, 'unit': None, 'rotation': None, 'size': None, 'template': None}
command :  -RK1


C:\Users\Admin\AppData\Local\Temp\ipykernel_10920\3433111061.py:49: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  if sum([token.similarity(nlp(word)[0]) > SIMILARITY_PARAMETER for word in EXPLICIT]) >= 1 :


In [334]:
def createSiteDict(dictSite : dict) -> dict:
    finalDict = {}
    keys = dictSite.keys()
    if "name" in keys and "orientation" in keys:
        finalDict["name"] = dictSite["name"]
        if dictSite["orientation"] !=None:
            finalDict["orientation"] = dictSite["orientation"]
    else: 
        raise Exception ("At least one mandatory parameters was forgotten")
    return finalDict
    

In [335]:
test = {"name" : "SITE", "orientation" : "ORIENTATION"}
print(createSiteDict(test))

{'name': 'SITE', 'orientation': 'ORIENTATION'}


In [322]:
def createBuildingDict(dictBuild : dict) -> dict : #NB COLOR + EXCEPTION FOR FUNCTION
    finalDict = {}
    keys = dictBuild.keys()
    print("name" in keys)
    if "name" in keys and "position" in keys and "rotation" in keys:
        finalDict["name"] = dictBuild["name"]
        finalDict["position"] = dictBuild["position"]
        finalDict["rotation"] = dictBuild["rotation"]
    else:
        raise Exception("Mandatory argument missing")
    if "size" in keys and "template" in keys:
        if dictBuild["size"] !=None and dictBuild["template"] !=None:
            raise Exception("A size and a template were given")
        elif dictBuild["size"] !=None:
            finalDict["size"] = dictBuild["size"]
        else:
            finalDict["template"] = dictBuild["template"]
    else:
        raise Exception("Neither a size neither a template were given")
    return finalDict

In [323]:
text = {'name': 'IMT', 'position': [0.0, 0.0], 'rotation': '90', 'size': None, 'template': "temp"}

print(createBuildingDict(text))

True
{'name': 'IMT', 'position': [0.0, 0.0], 'rotation': '90', 'template': 'temp'}


In [77]:
def createRoomDict(dictRoom : dict) -> dict:
    finalDict = {}
    keys = dictRoom.keys()
    if "name" in keys and "position" in keys and "rotation" in keys:
        finalDict["name"] = dictRoom["name"]
        finalDict["position"] = dictRoom["position"]
        finalDict["rotation"] = dictRoom["rotation"]
    else:
        raise Exception("At least one mandatory element is missing")
    
    if "size" in keys and "template" in keys:
        if dictRoom["size"] !=None and dictRoom["template"] !=None:
            raise Exception("A size and a template were given")
        elif dictRoom["size"] !=None:
            #If we have a size, we have to have an axisOrientation
            if "axisOrientation" in keys and dictRoom["axisOrientation"] !=None:
                finalDict["size"] = dictRoom["size"]
                finalDict["axisOrientation"] = dictRoom["axisOrientation"]
            else :
                raise Exception("axisOrientation missing while a size was given")
            if "floorUnit" in keys and dictRoom["floorUnit"] !=None:
                finalDict["floorUnit"] = dictRoom["floorUnit"]
        else:
            finalDict["template"] = dictRoom["template"]
    else:
        raise Exception("Neither a size neither a template were given")
    return finalDict


In [146]:
test = {'name': 'ROOM1', 'position': [4.0, 14.0], 'rotation': 10.0, 'size': [10.0, 10.0, 10.0], 'axisOrientation': '-x-y', 'floorUnit': 'm', 'template': None}
print(createRoomDict(test))

{'name': 'ROOM1', 'position': [4.0, 14.0], 'rotation': 10.0, 'size': [10.0, 10.0, 10.0], 'axisOrientation': '-x-y', 'floorUnit': 'm'}


In [78]:
def createRackDict(dictRack : dict) -> dict:
    finalDict = {}
    keys = dictRack.keys()
    if "name" in keys and "position" in keys and "rotation" in keys and 'unit' in keys:
        finalDict["name"] = dictRack["name"]
        finalDict["position"] = dictRack["position"]
        finalDict["unit"] = dictRack["unit"]
        finalDict["rotation"] = dictRack["rotation"]
    else:
        raise Exception("At least one mandatory element is missing")
    
    if "size" in keys and "template" in keys:
        if dictRack["size"] !=None and dictRack["template"] !=None:
            raise Exception("A size and a template were given")
        elif dictRack["size"] !=None:
            finalDict["size"] = dictRack["size"]
        else:
            finalDict["template"] = dictRack["template"]
    else:
        raise Exception("Neither a size neither a template were given")
    return finalDict

In [406]:
test = {'name': 'RACK1', 'position': [10.0, -10.0, 50.0], 'unit': 'tilde', 'rotation': ['0', '0', '180'], 'size': [10.0, 20.0, 30.0], 'template': None}
print(createRackDict(test))

{'name': 'RACK1', 'position': [10.0, -10.0, 50.0], 'rotation': ['0', '0', '180'], 'unit': 'tilde', 'size': [10.0, 20.0, 30.0]}


In [79]:
def createDeviceDict(dictDevice : dict) -> dict:
    finalDict = {}
    keys = dictDevice.keys()
    if "name" in keys:
        finalDict["name"] = dictDevice["name"]
    else:
        raise Exception("At least one mandatory element is missing")
    #A devive has either a position either a slot
    if "position" in keys and "slot" in keys:
        if dictDevice["position"] !=None and dictDevice["slot"] !=None:
            raise Exception("a posU and a slot were given")
        elif dictDevice["position"] !=None:
            finalDict["position"] = dictDevice["position"]
        else:
            finalDict["slot"] = dictDevice["slot"]
    #A device has either a size either a template
    if "size" in keys and "template" in keys:
        if dictDevice["size"] !=None and dictDevice["template"] !=None:
            raise Exception("A size and a template were given")
        elif dictDevice["size"] !=None:
            finalDict["size"] = dictDevice["size"]
        else:
            finalDict["template"] = dictDevice["template"]
    else:
        raise Exception("Neither a size neither a template were given")
    
    #A device can have a side if it has a template and a slot
    if dictDevice["template"] !=None and dictDevice["slot"] !=None:
        if side in keys and dictDevice["side"] !=None:
            finalDict["side"] = dictDevice["side"]

    return finalDict

In [411]:
test = {'name': 'DEVICE1', 'position': [4.0], 'size': [45.0], 'template': None, 'slot': None, 'side': None}
print(createDeviceDict(test))

{'name': 'DEVICE1', 'position': [4.0], 'size': [45.0]}


In [160]:
def createCorridorDict(dictCorridor : dict) -> dict:
    finalDict = {}
    keys = dictCorridor.keys()
    if "name" in keys and "position" in keys and "rotation" in keys and "unit" in keys and "size" in keys and "temperature" in keys:
        finalDict["name"] = dictCorridor["name"]
        finalDict["position"] = dictCorridor["position"]
        finalDict["unit"] = dictCorridor["unit"]
        finalDict["rotation"] = dictCorridor["rotation"]
        finalDict["size"] = dictCorridor["size"]
        finalDict["temperature"] = dictCorridor["temperature"]
    else:
        raise Exception("At least one mandatory argument was forgotten")
    return finalDict

In [169]:
def createTagDict(dictTag: dict) -> dict:
    finalDict={}
    keys = dictTag.keys()
    if "name" in keys and "color" in keys:
        finalDict["name"] = dictTag["name"]
        finalDict["color"] = dictTag["color"]
    else:
        raise Exception("At least one mandatory attribute was forgotten")
    return finalDict

In [170]:
test = {'name': 'CORR1', 'position': [0.0, 0.0], 'unit': 'metre', 'rotation': ['0', '0', '0'], 'size': [10.0, 10.0, 10.0], 'temperature': 'warm'}
print(createCorridorDict(test))
test = {'name': 'TAG1', 'color': '#154987'}
print(createTagDict(test))

{'name': 'CORR1', 'position': [0.0, 0.0], 'unit': 'metre', 'rotation': ['0', '0', '0'], 'size': [10.0, 10.0, 10.0], 'temperature': 'warm'}
{'name': 'TAG1', 'color': '#154987'}


In [293]:
def createSeparatorDict(dictEntity : dict) -> dict:
    finalDict = {}
    keys = dictEntity.keys()
    if "name" in keys and "startPosition" in keys and "endPosition" in keys and "wall" in keys:
        finalDict["name"] = dictEntity["name"]
        finalDict["startPosition"] = dictEntity["startPosition"]
        finalDict["endPosition"] = dictEntity["endPosition"]
        finalDict["wall"] = dictEntity["wall"]
    else:
        raise Exception("At least one mandatory argument was forgotten")
    return finalDict

In [295]:
test = {'name': 'SEPA1', 'startPosition': [10.0, 25.0], 'endPosition': [54.0, 78.0], 'wall': 'plain'}
print(createSeparatorDict(test))

{'name': 'SEPA1', 'startPosition': [10.0, 25.0], 'endPosition': [54.0, 78.0], 'wall': 'plain'}


In [296]:
def createPillarDict(dictEntity : dict) -> dict:
    finalDict = {}
    keys = dictEntity.keys()
    if "name" in keys and "position" in keys and "size" in keys and "rotation" in keys:
        finalDict["name"] = dictEntity["name"]
        finalDict["position"] = dictEntity["position"]
        finalDict["size"] = dictEntity["size"]
        finalDict["rotation"] = dictEntity["rotation"]
    else:
        raise Exception("At least one mandatory argument was forgotten")
    return finalDict

In [299]:
test = {'name': 'P1', 'position': [10.0, 21.0], 'size': [15.0, 78.0], 'rotation': 87.0}
print(createPillarDict(test))

{'name': 'P1', 'position': [10.0, 21.0], 'size': [15.0, 78.0], 'rotation': 87.0}


In [300]:
def createLabelDict(dictEntity: dict) -> dict:
    finalDict = {}
    keys = dictEntity.keys()
    if "name" in keys and "attribute" in keys:
        if dictEntity["name"] !=None and dictEntity["attribute"] !=None:
            raise Exception("A name and an attribute was given")
        if dictEntity["name"] !=None:
            finalDict["name"] = dictEntity["name"]
        elif dictEntity["attribute"] != None:
            finalDict["attribute"] =  dictEntity["attribute"]
        else:
            raise Exception("Both name and attribute are false")
    else:
        raise Exception("At least one mandatory argument was forgotten")
    return finalDict

In [306]:
test = {'attribute': None , 'font': None, 'background': None, 'name': 'NAME'}
print(createLabelDict(test))

{'name': 'NAME'}


In [364]:
def createDict(entity : str,dictEntity: dict) -> dict:
    match entity:
        case 'site': 
            return createSiteDict(dictEntity)
        case 'building':
            return createBuildingDict(dictEntity)
        case 'room':
            return createRoomDict(dictEntity)
        case 'rack':
            return createRackDict(dictEntity)
        case 'device':
            return createDeviceDict(dictEntity)
        case 'separator':
            return createSeparatorDict(dictEntity)
        case 'pillar':
            return createPillarDict(dictEntity)
        case 'label':
            return createLabelDict(dictEntity)
        case 'corridor':
            return createCorridorDict(dictEntity)
        case 'tag':
            return createTagDict(dictEntity)
    raise Exception("None of the entity were found")

In [307]:
from items.tools import *

In [309]:
test = {'name': 'ROOM1', 'position': [4.0, 14.0], 'rotation': 10.0, 'size': [10.0, 10.0, 10.0], 'axisOrientation': '-x-y', 'floorUnit': 'm', 'template': None}
print(createEntity("room", test))

The given parameters :
[True, True, True, True, True, True]
True
+bd:ROOM1@[4.0, 14.0]@10.0@-x-y@[10.0, 10.0, 10.0]@m


In [13]:
nlp("rotated")[0].similarity(nlp("rotation")[0])
# [nlp("current")[0].similarity(nlp(word)[0]) for word in ["axis", "orientation"]]

0.6624487638473511

In [14]:
#Test position, size, rotation, template
testNlp = nlp("at 0.0 0 create a building named BATIMENT with the dimension 50 40 60, turned by 90 degrees and with the template called FAKER")

print("Position :" + str(position(testNlp, 0, 0, 4, "building")))
print("Rotation :" + str(rotation(testNlp, 15, 11, 19, "building")))
print("Dimension :" + str(size(testNlp, 10, 6, 14, "building")))
print("Template :" + str(template(testNlp, 22, 19, 23, "building")))

TypeError: position() takes 4 positional arguments but 5 were given

In [63]:
spacy.displacy.render(testNlp, style="dep")